### Structured Output

Models can be requested to provide their response in a format matching a given schema. This is a useful for ensuring the ouput can be easily parsed and used by subsequent processing. Langchain supports multiple schema types and methods for enforcing structured output.

### Pydantic

Pydantic models provide the richest feature set with validation, description, and nested structures.

In [58]:
import os
from langchain.chat_models import init_chat_model
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

# model = init_chat_model("groq:openai/gpt-oss-120b")
model = init_chat_model("groq:qwen/qwen3-32b")

In [59]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The release year of movie")
    director: str = Field(description="The director of the movie")
    rating: float = Field(description="The rating of the movie out of 10")

In [60]:
model_with_structure = model.with_structured_output(Movie)

In [55]:
res = model_with_structure.invoke("provide details about movie titanic")
print(res)

title='Titanic' year=1997 director='James Cameron' rating=7.8


Message output alongside Parsed Structure

In [56]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The release year of movie")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The rating of the movie out of 10")

model_with_structure = model.with_structured_output(Movie, include_raw=True)

In [57]:
res = model_with_structure.invoke("provide details about movie titanic")
print(res)

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for details about the movie Titanic. Let me see. I need to use the Movie function provided. The required parameters are title, year, director, and rating. I remember Titanic was directed by James Cameron. The release year was 1997, I think. The rating is probably high, maybe around 7.8 or so. Let me double-check those details. Yep, director is James Cameron, year 1997, and the IMDb rating is 7.8. Alright, I'll structure the function call with those parameters.\n", 'tool_calls': [{'id': '5qrp53t49', 'function': {'arguments': '{"director":"James Cameron","rating":7.8,"title":"Titanic","year":1997}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 225, 'total_tokens': 392, 'completion_time': 0.286289921, 'completion_tokens_details': {'reasoning_tokens': 119}, 'prompt_time': 0.011082788, 'prompt_tokens_details': None, 'queue_

### Nested Structure

In [62]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="The budget of the movie in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)


In [64]:
response = model_with_structure.invoke("provide details about movie Inception")
print(response)

title='Inception' year=2010 cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne')] genres=['Science Fiction', 'Action'] budget=160.0


### TypedDict

TypedDict provieds a simple alternative using Python's built-in Typing, ideal when you don't need runtime validation.

In [70]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The release year of movie"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The rating of the movie out of 10"]

model_with_typeddict = model.with_structured_output(MovieDict)
res1 = model_with_typeddict.invoke("provide details about movie titanic")
print(res1)

{'director': 'James Cameron', 'rating': 7.8, 'title': 'Titanic', 'year': 1997}


In [71]:
class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="The budget of the movie in millions USD")

model_with_typeddict = model.with_structured_output(MovieDetails)
res2 = model_with_typeddict.invoke("provide details about movie titanic")
print(res2)


{'budget': 200000000, 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Jack Dawson'}, {'name': 'Kate Winslet', 'role': 'Rose DeWitt Bukater'}, {'name': 'Billy Zane', 'role': 'Caledon Hockley'}, {'name': 'Kathy Bates', 'role': 'Molly Brown'}, {'name': 'Frances Fisher', 'role': 'Ruth DeWitt Bukater'}], 'genres': ['Drama', 'Romance'], 'title': 'Titanic', 'year': 1997}


In [72]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### Data Classes

A data class typically contaning mainly data, although there aren't really any restrictinos. You create it using the @dataclass decorator.

In [73]:
import os
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')  

In [82]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

class ContactInfo(BaseModel):
    """Contact information for a person"""
    name: str = Field(description="Name of person")
    email: str = Field(description="Email of person")
    phone: str = Field(description="Phone number of person")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

model = create_agent(
    model = llm,
    response_format = ContactInfo    # Autoselecs provider stratergy
)

result = model.invoke({
    "messages":[{"role":"user", "content":"Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

print(result['structured_response'])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

@dataclass
class ContactInfo:
    """Contact information for a person"""
    name: str
    email: str
    phone: str

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)
 
agent = create_agent(
    model = llm,
    response_format = ContactInfo    # Autoselecs provider stratergy
)

result = agent.invoke({
    "messages":[{"role":"user", "content":"Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})  

print(result['structured_response'])

ImportError: Unable to import langchain_google_vertexai. Please install with `pip install -U langchain-google-vertexai`